In [7]:
try:
    import lazyarray
except:
    !pip install lazyarray
    !pip install hdbscan 
!pip install scipy --upgrade

    100% |████████████████████████████████| 24.7MB 1.4MB/s ta 0:00:011    50% |████████████████                | 12.4MB 4.0MB/s eta 0:00:04
Requirement not upgraded as not directly required: numpy>=1.8.2 in /opt/conda/lib/python3.5/site-packages (from scipy) (1.12.1)
cryptography 2.2.1 requires asn1crypto>=0.21.0, which is not installed.
cffi 1.11.5 requires pycparser, which is not installed.
allensdk 0.14.2 has requirement pandas<0.20.0,>=0.16.2, but you'll have pandas 0.23.1 which is incompatible.
  Found existing installation: scipy 1.1.0
    Uninstalling scipy-1.1.0:
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'trf_linear.py'
Consider using the `--user` option or check the permissions.

You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import os
import pickle

try:
    izhi_opt = pickle.load(open('protected/multi_objective_izhi.p','rb')) 
except:
    os.system('wget wget -O ncp_cell_layer_5_6.p https://osf.io/6yba2/download')
    os.system('wget -O multi_objective_izhi https://osf.io/3vp8d/download')
    izhi_opt = pickle.load(open('multi_objective_izhi.p','rb')) 


list(izhi_opt.values())[0]['pf'][0].dtc
list(izhi_opt.values())[0]['pf'][0].dtc.scores
list(izhi_opt.values())[0]['pf'][-1].dtc.scores
list(izhi_opt.values())[0]['pf'][2].dtc.scores
list(izhi_opt.values())[0]['pf'][1].dtc.scores
list(izhi_opt.values())[0]['pf'][1].dtc.vtest


{0: {'injected_square_current': {'amplitude': array(34.176063537597656) * pA,
   'delay': array(100.0) * ms,
   'duration': array(1000.0) * ms}},
 1: {'injected_square_current': {'amplitude': array(-10.0) * pA,
   'delay': array(200.0) * ms,
   'duration': array(500.0) * ms}},
 2: {'injected_square_current': {'amplitude': array(-10.0) * pA,
   'delay': array(200.0) * ms,
   'duration': array(500.0) * ms}},
 3: {'injected_square_current': {'amplitude': array(-10.0) * pA,
   'delay': array(200.0) * ms,
   'duration': array(500.0) * ms}},
 4: {'injected_square_current': {'amplitude': array(34.176063537597656) * pA,
   'delay': array(100.0) * ms,
   'duration': array(1000.0) * ms}},
 5: {'injected_square_current': {'amplitude': array(34.176063537597656) * pA,
   'delay': array(100.0) * ms,
   'duration': array(1000.0) * ms}},
 6: {'injected_square_current': {'amplitude': array(34.176063537597656) * pA,
   'delay': array(100.0) * ms,
   'duration': array(1000.0) * ms}}}

In [9]:
#from manager import ModelManager
#from tables import Cells, Model_Waveforms, Models, Cell_Closest_Ephyz_Clusters
import manager
reload(manager)
from manager import ModelManager


    

ImportError: bad magic number in 'manager': b'\x03\xf3\r\n'

In [ ]:

from pandas import DataFrame
import numpy as np
import pandas
from matplotlib import pyplot as plt

import string 
from scipy.stats import pearsonr
import statsmodels.formula.api as smf
import hdbscan
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', 20)
from manager import ModelManager
mgr = ModelManager()
mgr.server.connect()

In [ ]:
protocol_cost = {
    'Steady State':       1, # 1s
    'Standard':         5*2, # 5 repetitions of 1s SS 1s stim
    'Strong':           5*2, # 5 repetitions of 1s SS 1s strong stim
    'Input Resistance': 2*2, # 2 levels of 1s SS 1s stim
}

prop_protocol = {
    'AP1Amplitude': 'Standard',
    'AP2Amplitude': 'Standard',
    'AP12AmplitudeDrop': 'Standard',
    'AP12AmplitudeChangePercent': 'Standard',
    'AP1SSAmplitudeChange': 'Standard'  ,
    
    'AP1WidthHalfHeight': 'Standard',
    'AP2WidthHalfHeight': 'Standard',
    'AP12HalfWidthChangePercent': 'Standard',
    
    'AP1WidthPeakToTrough': 'Standard',
    'AP2WidthPeakToTrough': 'Standard',
    
    'AP1RateOfChangePeakToTrough': 'Standard',
    'AP2RateOfChangePeakToTrough': 'Standard', 
    'AP12RateOfChangePeakToTroughPercentChange': 'Standard',
    
    'AP1AHPDepth': 'Standard',
    'AP2AHPDepth': 'Standard',
    'AP12AHPDepthPercentChange': 'Standard',
    
    'AP1DelayMean': 'Standard',
    'AP2DelayMean': 'Standard',
    
    'AP1DelaySD': 'Standard',
    'AP2DelaySD': 'Standard',
    
    'AP1DelayMeanStrongStim': 'Strong',
    'AP2DelayMeanStrongStim': 'Strong',
    
    'AP1DelaySDStrongStim': 'Strong',
    'AP2DelaySDStrongStim': 'Strong',
    
    'Burst1ISIMean': 'Standard',
    'Burst1ISIMeanStrongStim': 'Strong',
    
    'Burst1ISISD': 'Standard',
    'Burst1ISISDStrongStim': 'Strong',
    
    'InitialAccommodationMean': 'Standard',
    'SSAccommodationMean': 'Standard',
    'AccommodationRateToSS': 'Standard',
    'AccommodationAtSSMean': 'Standard',
    'AccommodationRateMeanAtSS': 'Standard',
    
    
    'ISIMedian': 'Standard',
    'ISICV': 'Standard',
    'ISIBurstMeanChange': 'Standard',
    
    'SpikeRateStrongStim': 'Strong',
    
    'InputResistance': 'Input Resistance',
    
    'SteadyStateAPs': 'Steady State',
}

prop_names = [
    'AP1Amplitude',
    'AP2Amplitude',
    'AP12AmplitudeDrop',
    'AP12AmplitudeChangePercent',
    'AP1SSAmplitudeChange',  
    
    'AP1WidthHalfHeight',
    'AP2WidthHalfHeight',
    'AP12HalfWidthChangePercent',
    
    'AP1WidthPeakToTrough',
    'AP2WidthPeakToTrough',
    
    'AP1RateOfChangePeakToTrough',
    'AP2RateOfChangePeakToTrough',    
    'AP12RateOfChangePeakToTroughPercentChange',
    
    'AP1AHPDepth',
    'AP2AHPDepth',
    'AP12AHPDepthPercentChange',
    
    'AP1DelayMean',
    'AP2DelayMean',
    
    'AP1DelaySD',
    'AP2DelaySD',
    
    'AP1DelayMeanStrongStim',
    'AP2DelayMeanStrongStim',
    
    'AP1DelaySDStrongStim',
    'AP2DelaySDStrongStim',
    
    'Burst1ISIMean',
    'Burst1ISIMeanStrongStim',
    
    'Burst1ISISD',
    'Burst1ISISDStrongStim',
    
    'InitialAccommodationMean',
    'SSAccommodationMean',
    'AccommodationRateToSS',
    'AccommodationAtSSMean',
    'AccommodationRateMeanAtSS',
    
    
    'ISIMedian',
    'ISICV',
    'ISIBurstMeanChange',
    
    'SpikeRateStrongStim',
    
    'InputResistance',
    
    'SteadyStateAPs',
    
    'FrequencyPassAbove',
    'FrequencyPassBelow',
    
    'RampFirstSpike',
]